In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/CorpusforDDL_compute')
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CorpusforDDL_compute


In [2]:
import pandas as pd
import re
import unicodedata

# 國語日報新聞

In [3]:
# 讀入國語日報新聞 txt 檔
mandarin_daily_path = 'CorpusforDDL/國語日報新聞/MandarinDailyNews-20181217'
all_files = os.listdir(mandarin_daily_path)

mandarin_daily_corpus = []
for f in all_files:
  with open(f'{mandarin_daily_path}/{f}') as f:
      lines = f.readlines()
      mandarin_daily_corpus.append(lines)

In [4]:
# 前處理
def preprocess_mandarin_daily(string):

  url = r'(www|http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'

  clean_string = re.sub(url, '', string) # 移除 url
  clean_string = re.sub('[\n△◎]', '', clean_string) # 移除換行符號和△◎符號
  clean_string = re.sub("【\w+】", "", clean_string) # 移除【記者xxx報導】
  splitted_strings = re.split(r'(?<=[。！？])', clean_string) # 將長句子用。！？斷開
  splitted_strings = [x for x in splitted_strings if len(x)>1] # 濾掉空字串和只有一個標點符號的字串

  return splitted_strings

In [5]:
test = """
【記者楊惠芳報導】
△小說家徐薏藍每年固定出兩本書之外，還在春秋兩季，
◎邀約文友，組成旅行團，周遊列國，觀賞世界各地美景。
今年開春後，她準備率團去帛琉，一窺海底祕密。目前報名的有五對夫婦及六位文友。
欲參加文友請早報名，以免向隅。
http://www.lttc.ntu.edu.tw。
"""

print(preprocess_mandarin_daily(test))

['小說家徐薏藍每年固定出兩本書之外，還在春秋兩季，邀約文友，組成旅行團，周遊列國，觀賞世界各地美景。', '今年開春後，她準備率團去帛琉，一窺海底祕密。', '目前報名的有五對夫婦及六位文友。', '欲參加文友請早報名，以免向隅。']


In [12]:
mandarin_daily_corpus_preprocessed = []

for text in mandarin_daily_corpus:
  text_preprocessed = []
  for string in text:
    check_punctuation = re.search(r'[，。！？]', string) # 檢查字串是否包含，。！？
    check_chinese = re.search(r'[\u4e00-\u9fff]+', string) # 檢查字串是否包含中文字

    if check_punctuation and check_chinese:
      preprocessed = preprocess_mandarin_daily(string)
      text_preprocessed.append(preprocessed)

  mandarin_daily_corpus_preprocessed.append(text_preprocessed)

In [13]:
mandarin_daily_corpus_preprocessed[0][:5]

[['今天是中華民國九十年元旦，也是二十一世紀的第一天，陳水扁總統發表跨世紀談話，並且向全民祝賀新年。',
  '陳總統表示，過去政府的「戒急用忍」政策有當時的背景及其必要，但是未來政府將以「積極開放，有效管理」的新視野，規畫我們的新世紀經貿版圖。'],
 ['陳總統指出，二十一世紀，經濟競爭力依然是我們賴以生存的命脈。',
  '面對全球景氣循環的低潮以及包括美國、日本等國家經濟成長的向下修正，臺灣地區的經濟繁榮將再次面臨高難度的考驗，政府與民間要為最壞的可能做最好的準備。'],
 ['另外，因應我們即將加入世界貿易組織，陳總統說，政府有責任面對全新的經濟情勢，將包括兩岸經貿在內的各項課題，重新納入全球市場的考量。',
  '他說，在過去的「戒急用忍」之外，未來我們將以「積極開放，有效管理」的新視野，在知識經濟的既定方針之下，為新世紀經貿版圖做出宏觀的規畫，並且逐步加以落實。'],
 ['陳水扁總統說，面對過去半年多以來的政經情勢，國人應該都能深刻的體會，有穩定的政局才有經濟的繁榮，有穩定的政局也才有改革和進步。'],
 ['在兩岸問題方面，陳總統強調，依據中華民國憲法，「一個中國」原本並不是問題，也希望對岸能夠深入了解臺灣地區人民心中的疑慮在哪裡，尊重兩千三百多萬人民的意志。']]

In [14]:
len(mandarin_daily_corpus) == len(mandarin_daily_corpus_preprocessed)

True

In [21]:
# test
import pickle
mandarin_daily_test = [item for sublist in mandarin_daily_corpus_preprocessed[0] for item in sublist]
with open('pkl_files/compute_test/mandarin_daily_test.pkl', 'wb') as f:
    pickle.dump(mandarin_daily_test, f)

# ASBC

In [22]:
# 讀入 ASBC txt 檔
asbc_path = 'CorpusforDDL/ASBC_去XML標記'
all_files = os.listdir(asbc_path)

asbc_corpus = []
for f in all_files:
  with open(f'{asbc_path}/{f}') as f:
      lines = f.readlines()
      asbc_corpus.append(lines)

In [23]:
# 前處理
def preprocess_asbc(string):

  clean_string = unicodedata.normalize('NFKC', string) # 全形轉半形
  clean_string = re.sub(r'\n', '', clean_string) # 移除換行符號

  clean_string = clean_string.translate(str.maketrans({',': '，', 
                                                       '!': '！', 
                                                       '?': '？'})) # ，！？改回全形
  return clean_string

In [24]:
asbc_corpus_preprocessed = []

for text in asbc_corpus:
  preprocessed = [preprocess_asbc(string) for string in text]
  joined = ''.join(preprocessed)
  splitted = re.split(r'(?<=[。！？])', joined)
  asbc_corpus_preprocessed.append(splitted)

In [27]:
asbc_corpus_preprocessed[0][:5]

['碰到(VC) 這(Nep) 種(Nf) 情形(Na) ，(COMMACATEGORY)真(D) 是(SHI) 會(D) 令(VL) 人(Na) 又(Caa) 氣憤(VH) 又(Caa) 苦惱(VH) 。',
 '(PERIODCATEGORY)相信(VK) 大多數(Neqa) 的(DE) 母親(Na) 也(D) 都(D) 有(V_2) 過(Di) 類似(VG) 的(DE) 困擾(Na) ，(COMMACATEGORY)因為(Cbb) 我們(Nh) 的(DE) 生活(Na) 環境(Na) 逐漸(D) 被(P) 色情(Na) 電影(Na) 、(PAUSECATEGORY) 錄影帶(Na) 、(PAUSECATEGORY) 畫片(Na) 、(PAUSECATEGORY) 書刊(Na) 所(D) 污染(VC) 。',
 '(PERIODCATEGORY)尤其(D) 在(P) 「(PARENTHESISCATEGORY) 笑(VC) 貧(VH) 不(D) 笑(VA) 娼(Na) 」(PARENTHESISCATEGORY) 的(DE) 社會(Na) 風氣(Na) 裡(Ng) ，(COMMACATEGORY)有(V_2) 些(Nf) 唯利是圖(VH) 的(DE) 不法(A) 商人(Na) ，(COMMACATEGORY)更(D) 是(SHI) 以(P) 誇張(VHC) 的(DE) 色情(Na) 表演(Na) 和(Caa) 刺激(VH) 的(DE) 畫面(Na) 來(D) 招攬(VC) 生意(Na) 。',
 '(PERIODCATEGORY)要(D) 我們(Nh) 這些(Neqa) 做(VG) 父母(Na) 的(DE) 完全(D) 禁止(VE) 孩子(Na) 們(Na) 去(D) 接觸(VC) ，(COMMACATEGORY)可(D) 真(D) 是(SHI) 防不勝防(VH) 。',
 '(PERIODCATEGORY)縱使(Cbb) 你(Nh) 的(DE) 孩子(Na) 現在(Nd) 沒(D) 看(VC) 過(Di) ，(COMMACATEGORY)以後(Nd) 他(Nh) 同樣(VH) 會(D) 有(V_2) 機會(Na) 接觸(VC) ，(COMMACATEGORY)所以(Cbb) ，(COMMACATEGORY)我們(Nh) 要(D) 做(VC) 的(DE) 並(D

In [26]:
len(asbc_corpus) == len(asbc_corpus_preprocessed)

True

In [28]:
# test
with open('pkl_files/compute_test/asbc_test.pkl', 'wb') as f:
    pickle.dump(asbc_corpus_preprocessed[0], f)

# EDsplit

In [29]:
# 讀入 EDsplit txt 檔
edsplit_path = 'CorpusforDDL/EDsplit-1File1Days-DeleteRepeatPattern-TitleWithoutPeriod-out'
all_files = os.listdir(edsplit_path)[:10]

edsplit_corpus = []
for f in all_files:
  with open(f'{edsplit_path}/{f}', 'rb') as f:
      lines = f.readlines()
      edsplit_corpus.append(lines)

In [30]:
edsplit_corpus_decoded = []

for text in edsplit_corpus:
  text_decoded = []
  not_decoded = []
  for string in text:
    try:
      text = string.decode('cp950')
      text_decoded.append(text)
    except:
      not_decoded.append(string)
  edsplit_corpus_decoded.append(text_decoded)

In [31]:
# 前處理
def preprocess_edsplit(string):

  clean_string = unicodedata.normalize('NFKC', string) # 全形轉半形
  clean_string = re.sub(r'\r|\n', '', clean_string) # 移除換行符號

  clean_string = clean_string.translate(str.maketrans({',': '，', 
                                                       '!': '！', 
                                                       '?': '？'})) # ，！？改回全形
  clean_string = clean_string.strip()
  return clean_string

In [32]:
edsplit_corpus_preprocessed = []

for text in edsplit_corpus_decoded:
  preprocessed = [preprocess_edsplit(string) for string in text]
  joined = ''.join(preprocessed)
  splitted = re.split(r'(?<=[。！？])', joined)
  edsplit_corpus_preprocessed.append(splitted)

In [33]:
edsplit_corpus_preprocessed[0][:5]

['連接線(Na) 材廠(Nc) 報佳音(VA) 。',
 '(PERIODCATEGORY)連接線(Na) 材廠(Nc) 營運(VA) 傳(VD) 佳音(Na) ，(COMMACATEGORY)萬泰科(Nc) (6190)(Neu) 在(P) 接獲(VC) 俄羅斯(Nc) 訂單(Na) 後(Ng) ，(COMMACATEGORY)3月(Nd) 營收(Na) 以(P) 2(FW) ‧(PERIODCATEGORY) 13億(Neu) 元(Nf) 創下(VC) 單月(Nd) 歷史(Na) 新高(Na) ，(COMMACATEGORY)第一(Neu) 季(Nd) 稅前盈餘(Na) 4500萬(Neu) 元(Nf) ，(COMMACATEGORY)每(Nes) 股(Nf) 稅前盈餘(Na) 0(FW) ‧(PERIODCATEGORY) 5(FW) 元(Nf) ;(SEMICOLONCATEGORY)松(Na) 普(VH) (5488)(Neu) 獲得(VJ) 中國(Nc) TCL(FW) 的(DE) 追加(Nv) 訂單(Na) ，(COMMACATEGORY)第二(Neu) 季(Nd) 開始(VL) 出貨(VA) 。',
 '(PERIODCATEGORY)去年(Nd) 銅價(Na) 飆漲(VH) ，(COMMACATEGORY)連接線廠(Nc) 紛紛(D) 關門(VH) ，(COMMACATEGORY)使得(VL) 產業(Na) 出現(VH) 洗牌(VA) 效應(Na) ，(COMMACATEGORY)訂單(Na) 有(V_2) 向(P) 大廠(Nc) 集中(VC) 的(DE) 現象(Na) ，(COMMACATEGORY)這(Nep) 從(P) 網路線(Na) 大廠(Nc) 萬泰科(Nc) 的(DE) 營運(Nv) 可以(D) 看出(VE) 端倪(Na) ，(COMMACATEGORY)萬泰科(Nc) 在(P) 3月(Nd) 營收(Na) 就(D) 率先(D) 創下(VC) 新高(Na) ，(COMMACATEGORY)使得(VL) 今年(Nd) 獲利(Na) 將(D) 會(D) 創下(VC) 上櫃(VB) 來(Ng) 的(DE) 新高(Na) 。',
 '(PERIODCATEGORY)萬泰科(Nc) 3月(Nd) 營收(Na) 2(FW) ‧(PERIODCAT

In [34]:
len(edsplit_corpus_decoded) == len(edsplit_corpus_preprocessed)

True

In [35]:
# test
with open('pkl_files/compute_test/edsplit_test.pkl', 'wb') as f:
    pickle.dump(edsplit_corpus_preprocessed[0], f)

# Mandarin Daily News (小朋友作文)

In [36]:
# 讀入 Mandarin Daily News (小朋友作文)
essay_path = 'CorpusforDDL/MadarinDailyNews_from Zeng Yuanxian'
all_files = os.listdir(essay_path)

essay_corpus = []
for f in all_files:
  with open(f'{essay_path}/{f}', 'rb') as f:
      lines = f.readlines()
      essay_corpus.append(lines)

In [37]:
essay_corpus_decoded = []

for text in essay_corpus:
  text_decoded = []
  not_decoded = []
  for string in text:
    try:
      text = string.decode('utf-8')
      text_decoded.append(text)
    except:
      not_decoded.append(string)
  essay_corpus_decoded.append(text_decoded)

In [38]:
# 前處理
def preprocess_essay(string):

  clean_string = unicodedata.normalize('NFKC', string) # 全形轉半形
  clean_string = clean_string.translate(str.maketrans({',': '，', 
                                                       '!': '！', 
                                                       '?': '？'})) # ，！？改回全形
  clean_string = re.sub(r'.+\t.+\t.+\t', '', string) # 移除日期、題目、作者名
  clean_string = re.sub(r'。。', '', clean_string) # 移除多餘句號
  clean_string = re.sub(r'\r|\n', '', clean_string) # 移除換行符號

  return clean_string

In [39]:
essay_corpus_preprocessed = []

for text in essay_corpus_decoded:
  preprocessed = [preprocess_essay(string) for string in text]
  joined = ''.join(preprocessed)
  splitted = re.split(r'(?<=[。！？])', joined)
  essay_corpus_preprocessed.append(splitted)

In [40]:
essay_corpus_preprocessed[0][:5]

['新的開始，從心開始！',
 '我要改掉自己的壞脾氣，我常為了芝麻綠豆大的小事而生氣，實在是太不應該了。',
 '從現在開始，我要讓自己更有肚量，有話好好說，笑口常開，讓我的人緣越來越好。',
 '媽媽帶哥哥和我一起去菜園捉菜蟲。',
 '菜蟲吃的是菜，所以身體的保護色是綠色的，要仔細檢查才能發現。']

In [41]:
len(essay_corpus_decoded) == len(essay_corpus_preprocessed)

True

In [42]:
# test
with open('pkl_files/compute_test/essay_test.pkl', 'wb') as f:
    pickle.dump(essay_corpus_preprocessed[0], f)

# naer_written

In [45]:
naer_written_path = 'CorpusforDDL/naer/naer_written'
all_files = os.listdir(naer_written_path)

In [69]:
import xml.etree.ElementTree as ET
tree = ET.parse(f'{naer_written_path}/{all_files[9]}')
root = tree.getroot()

In [70]:
text = [x.text for x in root.iter('sentence')]
text[0]

'因此(Cbb)\u3000說(VE)\u3000智慧(Na)\u3000為(VG)\u3000「(PUNC)\u3000愚(VH)\u3000之(DE)\u3000始(Na)\u3000」(PUNC)\u3000是(SHI)\u3000言(VE)\u3000之(Nh)\u3000有(V_2)\u3000據(Na)\u3000的(T)\u3000。(PUNC)'